<img width="50px" align="left" style="margin-right:20px" src="http://data.cluster-lab.com/public-newprolab-com/npl_logo.png"> <b>New Professions Lab</b> <br /> Специалист по большим данным

# Проект 1

# Спрогнозировать пол и возрастную категорию интернет-пользователей по логу посещения сайтов

<img width="110px" align="left" src="http://data.cluster-lab.com/public-newprolab-com/project01_img0.png?img">

Одна из задач DMP-системы состоит в том, чтобы по разрозненным даннным, таким, как посещения неким пользователем сайтов, классифицировать его и присвоить ему определённую категорию: пол, возраст, интересы и так далее. В дальнейшем составляется портрет, или профиль, пользователя, на основе которого ему более таргетированно показывается реклама в интернете.

### Задача

Используя доступный набор данных о посещении страниц у одной части пользователей, сделать прогноз относительно **пола и возрастной категории** другой части пользователей. Угадывание (hit) - правильное предсказание и пола, и возрастной категории одновременно.

Мы не ограничиваем вас в выборе инструментов и методов работы с данными. Используйте любые эвристики, внешние источники, парсинг контента страниц — всё, что поможет вам выполнить задачу. Единственное ограничение — никаких ручных действий. Руками проставлять классы нельзя.

Поскольку это ваш проект, который мы наверняка захотите показать другим, уделите его оформлению достаточно времени. Мы рекомендуем сделать весь проект в этом ноутбуке. Снизу, под заданием, вы сможете описать ваше решение.

⏰ **Дедлайн: 16 мая 2019, 23:59**

### Это - не обычная лабораторка

Это - соревнование! В нем есть доска лидеров, которую можно увидеть после первой успешной проверки.

Но это так же и не обычное соревнование! Вы не проверяете ваш результат (предсказания), вместо этого проверочный скрипт запускает вашу программу, дает ей на стандартный ввод проверочные данные, и принимает предсказания от вашей программы на стандартном выводе.

Внимательно прочитайте секции "Обработка тестовых данных и формат вывода результатов" и "Проверка" ниже.

In [2]:
# Импортируем предложенные пакеты
import numpy as np
import pandas as pd
import re
import os, sys
import json
import pickle
from urllib.parse import urlparse, unquote

In [3]:
# Добавляем свои пакеты
from pandas.io.json import json_normalize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

## Обработка данных на вход

Для выполнения работы вам следует тспользовать файл `/data/share/project01/gender_age_dataset.txt`

In [4]:
file_path = '/data/share/project01/gender_age_dataset.txt'

Он содержит данные о посещении сайтов ~40 000 пользователей, при этом по некоторым из них (~ 35 000) известны их пол и возрастная категория, а по 5 000 - эта информация не известна. В файле есть 4 поля:
* **gender** - пол, принимающий значения `M` (male - мужчина), `F` (female - женщина), `-` (пол неизвестен);
* **age** - возраст, представленный в виде диапазона x-y (строковый тип), или `-` (возрастная категория неизвестна);
* **uid** - идентификатор пользователя, строковая переменная;
* **user_json** - поле json, в котором содержатся записи о посещении сайтов этим пользователем `(url, timestamp)`.

Первое, что обычно делают в таких случаях, — исследуют имеющийся датасет и разбираются, какие же данные мы получили.

Загрузим весь датасет в pandas:

In [5]:
%time df = pd.read_csv(file_path, sep='\t')

CPU times: user 7.16 s, sys: 916 ms, total: 8.08 s
Wall time: 8.07 s


И теперь попробуем понять, что у нас есть:

In [6]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


Что содержится в `user_json`?

In [7]:
df.iloc[0].user_json

'{"visits": [{"url": "http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun", "timestamp": 1419688144068}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426666298001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426666298000}, {"url": "http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story", "timestamp": 1426661722001}, {"url": "http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html", "timestamp": 1426661722000}]}'

Видим, что это некая сериализованная json-строка, которую можно легко разобрать через модуль `json`:

In [8]:
json.loads(df.iloc[0].user_json)

{'visits': [{'url': 'http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun',
   'timestamp': 1419688144068},
  {'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story',
   'timestamp': 1426666298001},
  {'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html',
   'timestamp': 1426666298000},
  {'url': 'http://news.yandex.ru/yandsearch?cl4url=chezasite.com/htc/htc-one-m9-delay-86327.html&lr=213&rpt=story',
   'timestamp': 1426661722001},
  {'url': 'http://www.sotovik.ru/news/240283-htc-one-m9-zaderzhivaetsja.html',
   'timestamp': 1426661722000}]}

Методом `pandas.DataFrame.apply` (хотя не только им) можно применить операцию десериализации json-строк ко всему датасету. Рекомендуем почитать [документацию по методу apply](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

Работая с подобными операциями, обратите внимание на kwargs-аргумент `axis`. Часто, забыв его указать, вы примените операцию не к ряду (строке), а к столбцу, что вряд ли входит в ваши планы.

### _More EDA_

In [9]:
# Review age column
df.age.value_counts()

25-34    15457
35-44     9360
-         5000
18-24     4898
45-54     4744
>=55      1679
Name: age, dtype: int64

In [10]:
# Review gender column
df.gender.value_counts()

M    18698
F    17440
-     5000
Name: gender, dtype: int64

# ПОДХОДЫ

__URL structure:__ scheme://netloc/path?query

__URL example:__ http://zebra-zoya.ru/200028-chehol-organayzer-dlja-macbook-11-grid-it.html?utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun

| Подходы | Основная идея | Цепочка | Gender | Age | Результат |
| :--- | :--- | :--- | :--- | :--- | :--- |
| Cамый первый заход | Выстроить цепочку 'dataset' -> 'submission' | 'Standard' url2domain -> netlocs -> CountVectorizer -> bag of words -> RandomForest (100, max_depth=None) | 0.669 | 0.428 | >0.28 |
| Улучшаем результат | XGBoost нам поможет | 'Standard' url2domain -> netlocs -> Count(Tfidf)Vectorizer -> bag of words -> **XGBoost (gender: 1000x6; age: 300x3)** | >0.68 | >0.44 | до 0.30 |
| Пробуем другие модели | Что сможет embedding+LSTM? | 'Standard' url2domain -> netlocs -> **KerasTokenizer -> Embedding -> LSTM** | - | - | долго/ нет смысла |
| Пробуем ещё модели | Что сможет обычная нейронная сеть? | 'Standard' url2domain -> netlocs -> KerasTokenizer -> **bag of words -> Dense Net** | - | - | overfitting/ мало данных |
| Прорыв с новыми данными | Пришло понимание, что только данные нам помогут | **'Modified'** url2domain -> netlocs + **path/query** -> CountVectorizer -> bag of words -> XGBoost | >0.70 | >0.44 | >0.31 |
| Попытка 'улучшить' данные | Add stop words, remove all digits, get hours as separate features, apply stemming, etc | 'Modified' url2domain + **timestamps** -> netlocs/path/query + **hour** -> **words pre-processing** -> CountVectorizer -> bag of words -> XGBoost | - | - | tradeoff gender vs age |
| 'Кручининский' прорыв | Пришло понимание, что фильтр на test_df дает прорыв в результате | 'Modified' url2domain -> netlocs/path/query  -> CountVectorizer -> bag of words -> XGBoost + **test_df фильтруем по медиане кол-ва слов** | >0.70 | >0.44 | >0.36 |
| Финализация | Отказ от stop words, remove digits, hours as features. Вместо этого hours added to bag of words + model tuning | **'Final'** url2domain -> netlocs/path/query -> CountVectorizer **(binary, min_df=6)** -> bag of words -> XGBoost **(gender: 1400x6; age: 350x4)** | ~0.705 | ~0.445 | ~0.37 val/ ~0.36 test |

## Очистка данных и feature engineering

Очистка данных и генерация новых фич составит значительную часть вашей работы. Именно здесь вы и должны продемонстрировать знания и креативность: чем лучше окажутся ваши фичи и чем лучше сможете убрать шум из датасета, тем лучших результатов вы достигнете.

Одна из первых вещей, которые можно попробовать — это вытащить домены и использовать их в качестве признаков. Можно воспользоваться функцией:

In [11]:
def url2domain(url):
    url = re.sub('(http(s)*://)+', 'http://', url)
    parsed_url = urlparse(unquote(url.strip()))
    if parsed_url.scheme not in ['http','https']:
        return None
    netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
    path = re.findall("([a-z]+[a-z0-9]+)", parsed_url.path)
    query = re.findall("([a-z]+[a-z0-9]+)", parsed_url.query)
    if path is not None:
        path = " ".join(path)
        result = netloc + "/" + path
    else:
        result = netloc
    if query is not None:
        query = " ".join(query)
        result = result + "?" + query
    if result is not None:
        return result        #str(result.encode('utf8')).strip()
    return None

Поскольку эта часть и есть ваша работа, мы не станем раскрывать все секреты (хотя несколько советов мы всё же дали, посмотрите ниже в разделе Подсказки).

### _Сделаем из записей о посещении сайтов документы для Bag of words_

In [12]:
# Преобразуем user_json в питоновские словари и нормализуем один уровень
%time df['visits'] = json_normalize(df.user_json.apply(json.loads))
df.head()

CPU times: user 33.1 s, sys: 2.55 s, total: 35.7 s
Wall time: 35.7 s


,gender,age,uid,user_json,visits
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000...",[{'url': 'http://zebra-zoya.ru/200028-chehol-o...
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=...","[{'url': 'http://sweetrading.ru/?p=900', 'time..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr...",[{'url': 'http://ru.oriflame.com/products/prod...
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r...",[{'url': 'http://translate-tattoo.ru/font-sele...
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#...","[{'url': 'https://mail.rambler.ru/#/folder/', ..."


#### Extract info from 'url' and 'timestamp'

In [13]:
# Вытаскиваем domains из списка словарей в список domains
%time df['domains'] = df['visits'].apply(lambda x: [url2domain(item['url']) + "/" + \
                                                    str(pd.Timestamp(item['timestamp'], unit='ms').hour) for item in x])


df[['gender', 'age', 'domains']].head()

CPU times: user 1min 57s, sys: 1.04 s, total: 1min 58s
Wall time: 1min 58s


,gender,age,domains
0,F,18-24,[zebra-zoya.ru/chehol organayzer dlja macbook ...
1,M,25-34,"[sweetrading.ru/?/22, sweetrading.ru/?/22, swe..."
2,F,25-34,"[ru.oriflame.com/products product?code/18, ru...."
3,F,25-34,[translate-tattoo.ru/font selection?hash c573a...
4,M,>=55,"[mail.rambler.ru/?/8, news.rambler.ru/?/8, mai..."


In [15]:
df['domains'][0]

['zebra-zoya.ru/chehol organayzer dlja macbook grid it html?utm campaign utm content utm medium cpc utm source begun/13',
 'news.yandex.ru/yandsearch?cl4url chezasite com htc htc one m9 delay html lr rpt story/8',
 'sotovik.ru/news htc one m9 zaderzhivaetsja html?/8',
 'news.yandex.ru/yandsearch?cl4url chezasite com htc htc one m9 delay html lr rpt story/6',
 'sotovik.ru/news htc one m9 zaderzhivaetsja html?/6']

In [16]:
# Делаем из списка domains строки для Bag of words
%time df['domains'] = df['domains'].apply(lambda x:' '.join(x))
df[['gender', 'age', 'domains']].head()

CPU times: user 300 ms, sys: 264 ms, total: 564 ms
Wall time: 562 ms


,gender,age,domains
0,F,18-24,zebra-zoya.ru/chehol organayzer dlja macbook g...
1,M,25-34,sweetrading.ru/?/22 sweetrading.ru/?/22 sweetr...
2,F,25-34,ru.oriflame.com/products product?code/18 ru.or...
3,F,25-34,translate-tattoo.ru/font selection?hash c573a5...
4,M,>=55,mail.rambler.ru/?/8 news.rambler.ru/?/8 mail.r...


### Теперь можно удалить ненужные колонки

In [18]:
df.columns

Index(['gender', 'age', 'uid', 'user_json', 'visits', 'domains'], dtype='object')

In [19]:
# можно удалить ненужные колонки
df.drop(['user_json',], axis=1, inplace=True)
df.columns

Index(['gender', 'age', 'uid', 'visits', 'domains'], dtype='object')

### Деление на train и test сеты, обучение модели, предсказания для test-сета

Давайте теперь оценим размер нашего train и test сетов. Train set:

In [20]:
len(df[~((df.gender == '-') & (df.age == '-'))])

36138

Test set:

In [21]:
len(df[(df.gender == '-') & (df.age == '-')])

5000

In [22]:
len(df) # Весь датасет

41138

Когда вы очистили данные и сгенерировали признаки, которые можно дать на вход алгоритму, следующий этап — это разделить данные на тренировочную и тестовую выборки. Сохраните train и test выборки в отдельных файлах, используя метод `pandas.DataFrame.to_csv`. Либо просто сделайте два датафрейма: `train_df` и `test_df`. Обучите модель на ваш выбор, оцените результат, подумайте, как можно его улучшить.

In [23]:
train_df = df.query("~(gender=='-' & age=='-')")
test_df = df.query("gender=='-' & age=='-'")
train_df.shape, test_df.shape

((36138, 5), (5000, 5))

## Preview CountVectorizer

In [24]:
vectorizer1 = CountVectorizer(binary=True, min_df=6)
%time vectorizer1.fit(train_df.domains)

CPU times: user 33.8 s, sys: 884 ms, total: 34.7 s
Wall time: 34.7 s


CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=6,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [25]:
vocabulary = set(vectorizer1.vocabulary_.keys())
len(vocabulary)

76775

## Review Test Data

In [27]:
test_df.sort_values(by='domains')[:10]

,gender,age,uid,visits,domains
36206,-,-,bdb2df89-4bcd-410f-8388-e18af8bfc6a4,[{'url': 'http://1-veda.info/india/gokarna-kar...,1-veda.info/india gokarna karnataka html?/9 ik...
40263,-,-,b28429bd-ca51-4553-8259-56137cdd7ec5,"[{'url': 'http://1000cxem.com/?', 'timestamp':...",1000cxem.com/?/12 yandex.ru/clck jsredir?from ...
39243,-,-,b19ecd12-40bc-4fc4-8a3d-bf25f80eebd3,[{'url': 'http://www.1000v1.ru/catalog/163/det...,1000v1.ru/catalog detail?/20 1000v1.ru/catalog...
38424,-,-,dcb9a4d4-8997-4791-8845-54119e0ab4b5,[{'url': 'http://www.1000v1.ru/catalog/49/deta...,1000v1.ru/catalog detail?/9 1000v1.ru/catalog ...
39120,-,-,b1013d93-f859-4e8f-8d6c-0b156acc0bbc,[{'url': 'http://www.1001eda.com/dieta-pri-pan...,1001eda.com/dieta pri pankreatite?/10 wh-lady....
38167,-,-,ed86a278-2715-4672-b10f-ced066e19b0f,[{'url': 'http://www.1001eda.com/grechka-v-gor...,1001eda.com/grechka gorshochkax?/11 mixiplay.c...
39199,-,-,b1633d95-0532-4d5a-a20c-05a6e6600642,[{'url': 'http://www.1001eda.com/gribnoj-rulet...,1001eda.com/gribnoj rulet lavashe?/12 pillsman...
37500,-,-,0278ec6f-c9be-4896-8606-6b03f9ff5a04,[{'url': 'http://www.1001eda.com/kofejnyj-keks...,1001eda.com/kofejnyj keks multivarke?/5 yandex...
36467,-,-,47d1f472-4c3d-4914-8e55-2b726de345cd,[{'url': 'http://www.1001eda.com/plov-v-multiv...,1001eda.com/plov multivarke recept poshagovym ...
40114,-,-,89f3f966-800f-4ac4-908e-9708e916a561,[{'url': 'http://www.1001eda.com/prostoj-tort-...,1001eda.com/prostoj tort iz gotovyx biskvitnyx...


In [28]:
test_df.domains.apply(lambda x: len(x.split(" "))).median()

85.0

In [29]:
%time test_df['count'] = test_df.domains.apply(lambda x: len([domain for domain in x.split()]))
test_df.head(15)

CPU times: user 724 ms, sys: 240 ms, total: 964 ms
Wall time: 974 ms


,gender,age,uid,visits,domains,count
36138,-,-,bd7a30e1-a25d-4cbf-a03f-61748cbe540e,[{'url': 'http://www.interfax.ru/business/4146...,interfax.ru/business?/14 amerikan-gruzovik.ru/...,10432
36139,-,-,bd7a6f52-45db-49bf-90f2-a3b07a9b7bcd,[{'url': 'https://www.packagetrackr.com/track/...,packagetrackr.com/track ups?/6 packagetrackr.c...,7695
36140,-,-,bd7a7fd9-ab06-42f5-bf0f-1cbb0463004c,[{'url': 'http://www.mk.ru/incident/2015/02/27...,mk.ru/incident latviya podnyala vozdukh istreb...,58
36141,-,-,bd7c5d7a-0def-41d1-895f-fdb96c56c2d4,[{'url': 'http://www.24open.ru/user/elena80204...,24open.ru/user elena8020445?/3 ohotniki.ru/wea...,526
36142,-,-,bd7e54a2-0215-45cb-a869-9efebf250e38,[{'url': 'http://www.dns-shop.ru/catalog/i1728...,dns-shop.ru/catalog i172806 tverdotelnyj nakop...,77
36143,-,-,bd7e9797-4cdb-46e1-a540-f3ea010605ad,[{'url': 'http://news.meta.ua/cluster:41878362...,news.meta.ua/cluster sevastopole nashli dokhlu...,9
36144,-,-,bd7e9ec7-fb67-45eb-8ad3-209d01d15ae6,"[{'url': 'http://dynamobryansk.forum24.ru/', '...",dynamobryansk.forum24.ru/?/15,1
36145,-,-,bd8056df-cc25-4b63-bc12-a46f888baa49,[{'url': 'http://www.2mm.ru/mzdorovie/634/gerp...,2mm.ru/mzdorovie gerpes vooruzhen ochen opasen...,14
36146,-,-,bd818690-73d2-445d-be5d-5c8f748dbb19,[{'url': 'http://www.lacywear.ru/goods/categor...,lacywear.ru/goods category?utm source yandex m...,49
36147,-,-,bd81e006-f059-4cdd-b716-3467c78d1312,"[{'url': 'http://nn.domru.ru/', 'timestamp': 1...",nn.domru.ru/?/16,1


In [30]:
test_df.query('count >= count.median()').shape

(2503, 6)

#  Обучение модели

Обучите модель на ваш выбор, оцените результат, подумайте, как можно его улучшить.

# XGBoost classifier

### _Predict Gender XGBoost_

| vectorizer | n_estimators | max_depth | lr | score | current leader
| :--- | :--- | --- | --- | --- | --- |
| CountVectorizer() | 1000 | 6 | 0.1 | 0.68545 |
| CountVectorizer(binary=True) | 1000 | 6 | 0.1 | 0.68612 |
| TfidfVectorizer(norm='l2') | 1000 | 6 | 0.1 | 0.68258 |
| TfidfVectorizer(norm=None) | 1000 | 6 | 0.1 | 0.68554 |
| TfidfVectorizer(binary=True, norm=None) + path | 1000 | 6 | 0.1 | 0.70170 | 
| 'hours' + 'words'/TfidfVectorizer(binary=True, norm=None) + path | 1000 | 6 | 0.1 | 0.70067 |
| 'hours' + 'words'/TfidfVectorizer(norm=None) + path | 1000 | 6 | 0.1 | 0.70167 |
| 'hours' + 'words'/CountVectorizer(binary=True, min_df=5) + path?query | 1000 | 6 | 0.1 | 0.70192 | 
| ~~'hours'~~ + 'words'/CountVectorizer(binary=True, min_df=5) + path?query | 1000 | 6 | 0.1 | 0.70349 | submitted
| 'words'/CountVectorizer(binary=True, min_df=5) + path?query + cyrillic | 1000 | 6 | 0.1 | 0.70264 | 
| 'words'/CountVectorizer(binary=True, min_df=5) + path?query + cyrillic + stemmer_eng_rus | 1000 | 6 | 0.1 | 0.70562 |
| 'words'/CountVectorizer(binary=True, min_df=5) + path?query + cyrillic + stemmer_rus | 1000 | 6 | 0.1 | 0.70668 | !!!
| 'words'/CountVectorizer(binary=True, min_df=5) + path?query + cyrillic + ~~stemmer_rus~~ ~~stop_words~~ | 1000 | 6 | 0.1 | 0.70239 | 
| 'words'/CountVectorizer(binary=True, min_df=5) + path?query + cyrillic + ~~stemmer_rus~~ + stop_words | 1000 | 6 | 0.1 | 0.70239 |
| 'words'/CountVectorizer(binary=True, min_df=5) + path?query + cyrillic + stemmer_rus + ~~stop_words~~ | 1000 | 6 | 0.1 | 0.70495 |
| 'words' as above + path?query + cyrillic + stemmer_rus + stop_words-middle | 1000 | 6 | 0.1 | 0.70507 |
| 'words' as above + path?query + cyrillic + stemmer_rus + stop_words-small+<15 | 1000 | 6 | 0.1 | 0.70356 |
| 'words' as above + path?query + cyrillic + stop_words-small | 1000 | 6 | 0.1 | 0.70280 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) | 1000 | 6 | 0.1 | 0.69743 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=5) | 1000 | 6 | 0.1 | 0.69610 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) | 1100 | 6 | 0.1 | 0.69754 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) | 1200 | 6 | 0.1 | 0.69821 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) | 1300 | 6 | 0.1 | 0.69893 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) | 1400 | 6 | 0.1 | 0.69840 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) | 1500 | 6 | 0.1 | 0.69818 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) | 1300 | 5 | 0.1 | 0.69650 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) | 1300 | 7 | 0.1 | 0.69657 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) | 1350 | 6 | 0.1 | 0.69810 |
| 'domains'/'full path'/'hour' + CountVect(binary, min_df=6) | 1300 | 6 | 0.1 | 0.70510 | !!!
| 'domains'/'full path' + 'query'/'hour' + CountVect(binary, min_df=6) | 1300 | 6 | 0.1 | 0.70421 | submitted
| 'domains'/'full path' + 'query'/'hour' + CountVect(binary, min_df=6) | 1200 | 6 | 0.1 | 0.70371 |
| 'domains'/'full path' + 'query'/'hour' + CountVect(binary, min_df=6) | 1400 | 6 | 0.1 | 0.70504 | submitted
| 'domains'/'full path' + 'query'/'hour' + CountVect(binary, min_df=6) | 1500 | 6 | 0.1 | 0.70457 |

In [32]:
p1 = Pipeline(steps=[
    ('vectorizer', CountVectorizer(binary=True, min_df=6)),
    ('clf_xgb1', xgb.XGBClassifier(max_depth=6, learning_rate=0.1, n_estimators=1400, verbosity=1, 
                                   objective='binary:logistic', n_jobs=-1))], verbose=True)

In [31]:
# fit the model and compute the cross-validation score
%time scores_xgb_gender = cross_val_score(p1, train_df['domains'].values, y=train_df['gender'].values, cv=3, n_jobs=-1, verbose=1)
scores_xgb_gender.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 56 concurrent workers.


CPU times: user 1.39 s, sys: 11.7 s, total: 13.1 s
Wall time: 5min 57s


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  6.0min finished


0.7037192041457807

In [33]:
# Fit the pipeline for saving its parameters
%time p1.fit(train_df['domains'].values, train_df['gender'].values)

[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=  35.3s
[Pipeline] .......... (step 2 of 2) Processing clf_xgb1, total= 2.1min
CPU times: user 1h 12min 44s, sys: 5min 29s, total: 1h 18min 14s
Wall time: 2min 39s


Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=True,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=6,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabular...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bytree=1, gamma=0,
                               learning_rate=0.1, max_delta_step=0, max_depth=6,
                               min_child_weight=1, missing=None,
                               n_estimators=

### _Predict Age XGBoost_

| vectorizer | n_estimators | max_depth | lr | score | current leader
| :--- | --- | --- | --- | --- | --- |
| CountVectorizer() | 300 | 3 | 0.1 | 0.44388 |
| CountVectorizer(binary=True) | 300 | 3 | 0.1 | 0.44385 |
| TfidfVectorizer() | 300 | 3 | 0.1 | 0.44227 |
| TfidfVectorizer(norm=None) | 300 | 3 | 0.1 | 0.44391 |
| TfidfVectorizer(norm=None)+stop_words | 300 | 4 | 0.1 | 0.44316 |
| TfidfVectorizer(norm=None)+stop_words | 300 | 3 | 0.1 | 0.44382 |
| TfidfVectorizer(binary=True, norm=None)+stop_words_ex_digits | 300 | 3 | 0.1 | 0.44333 |
| TfidfVectorizer(norm=None)+stop_words_ex_digits | 300 | 3 | 0.1 | 0.44333 |
| TfidfVectorizer(norm=None)+path+stop_words_ex_digits | 300 | 3 | 0.1 | 0.44197 |
| TfidfVectorizer(binary=True, norm=None)+path+stop_words_ex_digits | 300 | 4 | 0.1 | 0.44241 | 
| 'hours' + 'words'/TfidfVectorizer(norm=None)+path+stop_words_ex_digits | 300 | 4 | 0.1 | 0.44044 |
| 'hours' + 'words'/TfidfVectorizer(norm=None)+path+stop_words_ex_digits | 300 | 3 | 0.1 | 0.44147 |
| 'hours' + 'words'/CountVectorizer(binary=True, min_df=5)+path?query+stop_words | 300 | 3 | 0.1 | 0.44269 |
| 'hours' + 'words'/as above + OneVsRestClassifier(XGBClassifier) | 300 | 3 | 0.1 | 0.44255 | 
| 'hours' + 'words'/as above + OneVsRestClassifier(XGBClassifier) | 300 | 4 | 0.1 | 0.44391 |  
| ~~'hours'~~ + 'words'/as above + OneVsRestClassifier(XGBClassifier) | 300 | 4 | 0.1 | 0.44440 | submitted 
| 'words'/as above + OneVsRestClassifier(XGBClassifier) + cyrillic | 300 | 4 | 0.1 | 0.44446 | 
| 'words'/as above + OneVsRestClassifier(XGBClassifier) + cyrillic + ~~stop_words~~ | 300 | 4 | 0.1 | 0.44471 | 
| 'words'/as above + OneVsRestClassifier(XGBClassifier) + cyrillic + stop_words[small] | 300 | 4 | 0.1 | 0.44490 | !!!
| 'words'/as above + OneVsRestClassifier(XGBClassifier) + cyrillic + stemmer_rus +~~stop_words~~ | 300 | 4 | 0.1 | 0.44371 | 
| 'words'/as above + OneVsRestClassifier(XGBClassifier) + cyrillic + stemmer + stop_words-middle | 300 | 4 | 0.1 | 0.44316 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=5) + OneVsRest | 300 | 4 | 0.1 | 0.44476 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 300 | 4 | 0.1 | 0.44476 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 400 | 4 | 0.1 | 0.44496 | 
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 500 | 4 | 0.1 | 0.44457 |
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 350 | 4 | 0.1 | 0.44509 | 
| 'domains'/'1st word path'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 370 | 4 | 0.1 | 0.44512 | 
| 'domains'/'full path'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 370 | 4 | 0.1 | 0.44388 | 
| 'domains'/'full path' + 'query'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 370 | 4 | 0.1 | 0.44523 | submitted 
| 'domains'/'full path' + 'query'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 300 | 4 | 0.1 | 0.44529 |
| 'domains'/'full path' + 'query'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 400 | 4 | 0.1 | 0.44551 | !!!
| 'domains'/'full path' + 'query'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 450 | 4 | 0.1 | 0.44548 |
| 'domains'/'full path' + 'query'/'hour' + CountVect(binary, min_df=6) + OneVsRest | 450 | 4 | 0.1 | 0.44518 |

In [42]:
p2 = Pipeline(steps=
              [('vectorizer', CountVectorizer(binary=True, min_df=6)),
               ('clf_xgb2', OneVsRestClassifier(xgb.XGBClassifier(max_depth=4, learning_rate=0.1, n_estimators=350, verbosity=1, 
                                              objective='binary:logistic', scale_pos_weight=1), n_jobs=-1)) 
              ], verbose=True)

In [38]:
# fit the model and compute the cross-validation score
%time scores_xgb_age = cross_val_score(p2, train_df['domains'].values, y=train_df['age'].values, cv=3, n_jobs=-1, verbose=1)
scores_xgb_age.mean()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 56 concurrent workers.


CPU times: user 1.22 s, sys: 8.7 s, total: 9.91 s
Wall time: 2min 24s


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.4min finished


0.4451816709900922

In [43]:
# Fit the pipeline for saving its parameters
%time p2.fit(train_df['domains'].values, train_df['age'].values)

[Pipeline] ........ (step 1 of 2) Processing vectorizer, total=  34.9s
[Pipeline] .......... (step 2 of 2) Processing clf_xgb2, total= 1.5min
CPU times: user 34 s, sys: 7.98 s, total: 42 s
Wall time: 2min 5s


Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=True,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=6,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabular...
                 OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5,
                                                             booster='gbtree',
                                                             colsample_bylevel=1,
                                                             colsample_bytree=1,
                 

### Сохранение модели

Обучать модель вы можете в ноутбуке - это удобно. А после того как модель обучилась и стала выдавать приемлемое качество на вашей валидационной выборке, сохраните ее в отдельном файле (например, используя pickle):

Однако, не забудьте сохранить и код генерации признаков и обучения модели - это нужно для воспроизводимости результатов.

### Обработка тестовых данных и формат вывода результатов

Помимо того, что у вас должна получиться точная модель, вам нужно уложиться в SLA (service-level agreement). Всё почти как по-настоящему. Результатом вашей работы в данном случае будет не выходной файл, в котором вы всё посчитали для скрытой выборки, а скрипт, который мы будем запускать и проверять SLA и точность.

Создайте в корне своей домашней директории файл `project01_gender-age.py`. 

Назначте ему нужные права: `chmod 755 project01_gender-age.py`.

Вот фрагмент кода, который считывает данные из потока стандартного ввода:

Далее вы должны применить все те же самые преобразования данных, которые (возможно) применили к обучающей выборке.
Затем считываете модель:

Путь к модели необходимо указывать относительно вашей домашней директории. Например, если в директории `name.surname` есть папка `project01`, в которой лежит модель `project01_model.pickle`, то путь к модели в скрипте будет выглядеть как `project01/project01_model.pickle`.

Задача вашего скрипта сделать предсказание по всем полученным строкам и выдать результат в формате json. В файле должны быть только те пользователи, у которых пол и возрастная категория изначально неизвестны, и они должны быть **отсортированы по UID по возрастанию значений лексикографически.** Пример вывода указан ниже.

Для самопроверки вы можете локально оттестировать свой скрипт, используя следующую команду:

### Подсказки

1. Есть много различных способов решить данную задачу: можно просто хорошо поработать с урлами и доменами, можно пропарсить содержимое этих урлов (заголовки, текст и т.д.) и воспользоваться неким векторизатором типа TF\*IDF для генерации дополнительных фич, которые уже в дальнейшем вы подадите на вход ML-алгоритму, можно сделать тематическое моделирование (LDA, BigARTM) сайтов и использовать одну или несколько тем в качестве фич.

2. Возможно, что данные грязные и их нужно дополнительно обработать. Спецсимволы, кириллические домены? Уделите этому этапу достаточно времени: здесь чистота датасета важнее, чем выбор алгоритма.

3. Часто бывает, что лучшее решение с точки зрения результата — оно же самое простое. Попробуйте сначала простые способы, простые алгоритмы, прежде чем переходить к тяжёлой артиллерии. Один из вариантов — начать с небольшого RandomForest.

4. Вам почти наверняка понадобится что-то из пакета sklearn. [Документация](http://scikit-learn.org/stable/user_guide.html) — ваш лучший друг.

5. Вы можете сначала предсказать пол, а затем возраст, либо сразу и то, и другое. Экспериментируйте.

6. Объединяйтесь в команды. Так гораздо веселее и интереснее.

### Проверка
Проверка осуществляется из [Личного кабинета](http://lk.newprolab.com/lab/project1). До дедлайна вы будете проверять работу своего скрипта на валидационной выборке (2 000 записей). При наступлении дедлайна мы автоматически пересчитаем модели по скрытой тестовой выборке (3 000 записей). Это и будет финальным результатом.

* В поле `part of users with predicted gender + age` - указана доля пользователей, которая была предсказана от общего числа неизвестных пользователей (пример: по 3 000 был сделан прогноз, а всего было неизвестно 5 000, чекер выдаст 0.6).

* В поле `correctly predicted users / total number of users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа всех пользователей (пример: по 3 000 был сделан прогноз, правильно было спрогнозировано 1 500, а всего было неизвестно 5 000, чекер выдаст 0.3)

* В поле `correctly predicted users / number of predicted users` - указана доля пользователей, которая была правильно предсказана (совпадает и пол, и возраст) от общего числа предсказанных пользователей (пример: по 3 000 был сделан прогноз, из них правильно предсказано 1 500, чекер выдаст 0.5).

**Важное замечание!** Вы должны дать прогноз хотя бы по 50% пользователей, у которых изначально не указан пол и возрастная категория. Иными словами, вы можете оставить неопределенными не более 50% изначально неопределенных пользователей.

**Если доля в последнем поле превысит порог 0.28, то проект будет засчитан, при условии что выполнен SLA в 0.04 секунды на каждого пользователя (т.е. на каждую строчку тестового датасета)**

Лучшей команде, набравшей максимальный результат, мы подарим специальный приз, о котором скажем позднее.

# APPENDIX

## Neural Network

## Gender

### Model with Embedding

### Model with Bag of words

### Age

#### Sklearn Pipeline Gender

#### Sklearn Pipeline age

# Random Forest classifier

### _Predict Gender Random Forest_

| vectorizer | n_estimators | max_depth | lr | cv_score | current_leader
| :--- | --- | --- | --- | --- | --- |
| CountVectorizer() | 100 | None | - | 0.66923 | !!!

### _Make Pipeline for gender prediction_

### _Predict Age Random Forest_

| vectorizer | n_estimators | max_depth | lr | cv_score | current_leader
| :--- | --- | --- | --- | --- | --- |
| CountVectorizer() | 100 | None | - | 0.42777 | !!!

### _Make Pipeline for Age prediction_